In [1]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

import argparse
from T5Finetunner import T5FineTuner, LoggingCallback
import glob
import random
import shutil
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from T5Finetunner import ImdbDataset 

data_path = "/media/zihao/New Volume1/UMASS/685_e/github/Zihao_branch/data/aclImdb_v1/aclImdb"


In [7]:
print(torch.cuda.is_available())

True


In [3]:
args_dict = dict(
    data_dir= data_path, # path for data files
    output_dir="checkpoints", # path to save the checkpoints
    model_name_or_path='t5-base',
    tokenizer_name_or_path='t5-base',
    max_seq_length=512,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=1,
    eval_batch_size=1,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)
args = argparse.Namespace(**args_dict)

In [5]:

tokenizer = T5Tokenizer.from_pretrained('t5-base')

dataset = ImdbDataset(tokenizer, data_path, 'test',  max_len=512)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

it = iter(loader)

batch = next(it)
batch["source_ids"].shape

device = torch.device("cuda")
model = T5FineTuner(args)
model = T5ForConditionalGeneration.from_pretrained('t5_base_imdb_sentiment')
# ckpt = torch.load('t5_imdb_sentiment/checkpointepoch=1.ckpt')
# #model = model.load_state_dict(torch.load("t5_base_imdb_sentiment/pytorch_model.bin"))
# model.load(ckpt['state_dict'])
model.to(device)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [16]:

outputs = []
targets = []
outs = model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

dec = [tokenizer.decode(ids) for ids in outs]

texts = [tokenizer.decode(ids) for ids in batch['source_ids']]
target = [tokenizer.decode(ids) for ids in batch['target_ids']]
dec = [tokenizer.decode(ids) for ids in outs]

outputs.extend(dec)
targets.extend(target)


print(targets)
print(outputs)
for i,t in enumerate(targets):
    targets[i] = t.replace('</s>','')
for i,t in enumerate(outputs):
    outputs[i] = t.replace('<pad> ','')
print(targets)
print(outputs)

print(metrics.accuracy_score(targets, outputs))
print(metrics.classification_report(targets, outputs))
    

['positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>', 'positive</s>']
['<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> negative', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', '<pad> positive', 

/home/zihao/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zihao/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zihao/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
loader = DataLoader(dataset, batch_size=32, num_workers=4)
model.eval()
outputs = []
targets = []
for batch in tqdm(loader):
  outs = model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

  dec = [tokenizer.decode(ids) for ids in outs]
  target = [tokenizer.decode(ids) for ids in batch["target_ids"]]
  
  
  
  for i,t in enumerate(target):
    target[i] = t.replace('</s>','')
  for i,t in enumerate(dec):
    dec[i] = t.replace('<pad> ','')
  
  outputs.extend(dec)
  targets.extend(target)
  
print(metrics.accuracy_score(targets, outputs))
print(metrics.classification_report(targets, outputs))

100%|██████████| 782/782 [15:52<00:00,  1.22s/it]


0.94976
              precision    recall  f1-score   support

    negative       0.96      0.94      0.95     12500
    positive       0.94      0.96      0.95     12500

    accuracy                           0.95     25000
   macro avg       0.95      0.95      0.95     25000
weighted avg       0.95      0.95      0.95     25000

